In [1]:
from boto.mws.connection import MWSConnection
import numpy as np
import pandas as pd
from itertools import zip_longest
import time
from attributes import Attributes
from boto.exception import BotoServerError
from multiprocessing import Pool
import multiprocessing as mp
from tqdm import *
import csv
import os

In [2]:
aws_access_key_id = os.environ.get('aws_access_key_id')
aws_secret_access_key = os.environ.get('aws_secret_access_key')
sellerid = os.environ.get('sellerid')
marketplaceid = os.environ.get('marketplaceid')
mws = MWSConnection(aws_access_key_id=aws_access_key_id,
                    aws_secret_access_key=aws_secret_access_key,
                    Merchant=sellerid)

data = pd.read_csv('goodstuff.csv.xz')

/home/lwgray/anaconda3/envs/ddl/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,6,13,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
asin_list = list(zip_longest(*[iter(data.asin.unique())]*10))
# asin_list = list(zip_longest(*[iter(asin_list)]*10))
asin_100 = asin_list[:100]
asin_2 = asin_list[:2]
asin_10 = asin_list[:10]

In [4]:
mws.aws_access_key_id
single = [asin_list[0][0]]
print(single)
mws.get_matching_product(MarketplaceId=marketplaceid, ASINList=single)

['B004QNL2CC']


GetMatchingProductResponse{'xmlns': 'http://mws.amazonservices.com/schema/Products/2011-10-01'}(GetMatchingProductResult: [ProductsBulkOperationResult{'ASIN': 'B004QNL2CC', 'status': 'Success'}(Product: Product{'xmlns': 'http://mws.amazonservices.com/schema/Products/2011-10-01', 'xmlns:ns2': 'http://mws.amazonservices.com/schema/Products/2011-10-01/default.xsd'}(Identifiers: ^Identifiers^{}(MarketplaceASIN: ^MarketplaceASIN^{}(MarketplaceId: 'ATVPDKIKX0DER', ASIN: 'B004QNL2CC'), SKUIdentifier: None), AttributeSets: ^AttributeSets^{}(ItemAttributes: [ItemAttributes{'xml:lang': 'en-US'}(Actor: [], Artist: [], Author: [], Creator: [], Director: [], Feature: [], Format: [], GemType: [], MaterialType: [], MediaType: [], OperatingSystem: [], Platform: [], ItemDimensions: 20.50inchesx23.00inchesx6.00inchesx5.80pounds, ListPrice: None, PackageDimensions: 5.90inchesx22.60inchesx19.40inchesx5.80pounds, SmallImage: Image{}(URL: 'http://ecx.images-amazon.com/images/I/4107b76lPWL._SL75_.jpg', Heigh

In [5]:
def get_attributes(asin_list):
    master = []
    for asins in asin_list:
        try:
            try:
                response = mws.get_matching_product(MarketplaceId=marketplaceid, ASINList=asins)
            except AttributeError:
                print('AttributeError')
                time.sleep(20)
                response = mws.get_matching_product(MarketplaceId=marketplaceid, ASINList=asins)
            except BotoServerError:
                print('BotoServerError')
                time.sleep(20)
                response = mws.get_matching_product(MarketplaceId=marketplaceid, ASINList=asins)
            except:
                print('Exception')
                raise
            for result in response._result:
                product = result.Product
                identifier = result['ASIN']
                try:
                    attributes = product.AttributeSets.ItemAttributes[0]
                except AttributeError:
                    continue
                dimensions = attributes.PackageDimensions
                a = Attributes(product, attributes, dimensions)
                master.append([identifier, a.get_brand(), a.get_length(), a.get_width(),
                       a.get_height(), a.get_salesrank(), a.get_manufacturer(), a.get_max_age(),
                       a.get_min_age(), a.get_product_group()])
        except:
            time.sleep(20)
    return master

In [12]:
pool = Pool(processes=6)
with open('test2.csv', 'a') as f:
    writer = csv.writer(f)
    for results in pool.imap_unordered(get_attributes, [asin_list[:1000]]):
        for result in results:
            writer.writerow(result)
pool.close()
pool.join()

#for res in pool.map(get_attributes, [asin_list]):
#    print(res)

In [55]:
attr = pd.DataFrame(result[0], columns=['id', 'brand', 'length', 'width', 'height',
                                    'salesrank', 'manufacturer', 'max_age',
                                    'min_age', 'product_group'])
attr.to_csv('attributes.csv', header=False)